In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../')

In [2]:
import torch
from src.models.ecg_step_module import EcgEmbed
from src.basic.dx_and_feat import Feature, zero_vec

N = 64 # batch_size
batched_ecg = torch.rand(N, 12, 5000)
batched_feat = zero_vec(Feature, [N])
batched_input = (batched_ecg, batched_feat)
all_mid_output = {'pipeline': {}}
embed = EcgEmbed(all_mid_output)(batched_input)
print(all_mid_output['EcgEmbed']['embed'].shape)

/home/ftian/storage/miniconda/envs/ecg_xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([64, 240])


In [3]:
# from torch import nn
# from pmlayer.torch.hierarchical_lattice_layer import HLattice

# output_layer = nn.Linear(64, 1)
# consequent_model = nn.Sequential(self.embed_layer, output_layer)
# HLattice(263, sizes, self.lattice_inc_indices, consequent_model)

In [16]:
from src.models.ecg_step_module import BlockModule
# all_mid_output['BlockModule'].clear()
block_module = BlockModule(all_mid_output)
# print(block_module)
block_module(batched_input)

torch.Size([64, 1])


(tensor(0., grad_fn=<BinaryCrossEntropyBackward0>),
 tensor([0.], grad_fn=<AddBackward0>))

In [17]:
print(all_mid_output['BlockModule']['LPR_imp'].shape)
print(all_mid_output['BlockModule']['LQRS_imp'].shape)
print(all_mid_output['BlockModule']['AVB'].shape)
print(all_mid_output['BlockModule']['LBBB'].shape)
print(all_mid_output['BlockModule']['RBBB'].shape)

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])


In [23]:
from src.basic.rule_ml import SeqSteps
from src.models.ecg_step_module import EcgEmbed, BlockModule
from src.basic.dx_and_feat import Feature, zero_vec

N = 64 # batch_size
batched_ecg = torch.rand(N, 12, 5000)
batched_feat = zero_vec(Feature, [N])
batched_input = (batched_ecg, batched_feat)
all_mid_output = {}

# test SeqSteps
seq1 = SeqSteps('seq1', all_mid_output, [EcgEmbed(all_mid_output), BlockModule(all_mid_output)])
losses = seq1(batched_input)
print(losses)

<class 'dict'>
<class 'dict'>
<class 'dict'>
(0, 0)


In [34]:
from src.basic.rule_ml import PipelineModule
from src.basic.dx_and_feat import Diagnosis
y = torch.zeros([N, len(Diagnosis)], dtype=torch.float32)
batch = (batched_input, y)
# test PipelineModule
pipeline_module = PipelineModule()
all_mid_output = pipeline_module.all_mid_output
seq2 = SeqSteps('seq2', all_mid_output, [EcgEmbed(all_mid_output), BlockModule(all_mid_output)])
pipeline_module.add_pipeline(seq2)
losses = pipeline_module.training_step(batch, 0)
print(losses)

tensor(0.1047, grad_fn=<AddBackward0>)
